# <center> **Implementation of Neural Network** </center>
#### <center> <span style="color:gray"> ***With Stocastic, Mini-Batch and Batch Gradient Descent*** </span> </center>

> Layers

In [1]:
import numpy as np
from abc import ABC, abstractmethod

In [2]:
class ActivationFunction(ABC):
    @abstractmethod
    def activate(self, input):
        pass
    
    @abstractmethod
    def gradient(self, input):
        pass

In [3]:
class SigmoidActivationFunction(ActivationFunction):
    def activate(self, input):
        return 1.0 / (1.0 + np.exp(-input))
    
    def gradient(self, input): # batc_size * number_of_inputs * number_of_inputs
        activated = self.activate(input)
        res =  (1 - activated) * activated
        
        grad = []
        for idx in range(input.shape[1]):
            grad.append(np.diag(res[:,idx]))

        return np.stack(grad)

In [ ]:
class LossFunction(ABC):
    @abstractmethod
    def loss(self, target, result):
        pass
    
    def gradient(self, target, result):
        pass

In [ ]:
class MeanSquareLoss(LossFunction):
    def loss(self, target, result):
        l = target - result
        l = l * l
        res = 0.5 * np.sum(l) / np.size(target)
        return res
    
    def gradient(self, target, result):
        return result - target

In [4]:
class Layer(ABC):
    def __init__(self) -> None:
        self._input = None
        self._output = None
        
    @property
    def output(self):
        if(self._output is None):
            print("Output is not calculated yet!")
        return self._output
    
    @abstractmethod
    def forward(self, input): # input size = (number_of_features X batch_size)
        pass
    
    @abstractmethod
    def backward(self, output_gradient, learning_rate : float):
        pass

In [5]:
class DenseLayer(Layer):
    def __init__(self, number_of_inputs : int, number_of_outputs : int) -> None:
        self._number_of_inputs = number_of_inputs
        self._number_of_outputs = number_of_outputs
        self._weight = np.random.randn(number_of_inputs, number_of_outputs)
        self._bias = np.random.randn(number_of_outputs, 1)
        super().__init__()
    
    def forward(self, input):
        self._input = input
        self._output = np.matmul(self._weight.T, input) + self._bias
        return self._output
    
    def backward(self, output_gradient, learning_rate: float):
        res =  np.matmul(self._weight, output_gradient)
        
        gradient_weight = np.matmul(self._input, output_gradient.T)
        gradient_bias = output_gradient
        self._weight -= learning_rate * gradient_weight
        self._bias -= learning_rate * gradient_bias

        return res

In [6]:
class ActivationLayer(Layer):
    def __init__(self, activation_function : ActivationFunction) -> None:
        super().__init__()
        self._activation_function = activation_function
        
    def forward(self, input):
        self._input = input
        self._output = self._activation_function.activate(self._input)
        return self._output
    
    def backward(self, output_gradient, learning_rate: float):
        output_gradient = np.transpose(output_gradient)
        output_gradient = np.expand_dims(output_gradient, axis=2)
        res = np.matmul(self._activation_function.gradient(self._input), output_gradient)
        res = np.squeeze(res)
        res = np.transpose(res)
        
        return res

In [ ]:
class NeuralNetwork:
    def __init__(self, ) -> None:
        self.layers : list[Layer] = []
        self.loss_function